# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2023 1:43PM,264911,16,9578462,Sartorius Bioprocess Solutions,Released
1,Jun 16 2023 1:41PM,264910,10,SONSB0002233,"Nextsource Biotechnology, LLC",Released
2,Jun 16 2023 1:38PM,264909,10,MSP222764,"Methapharm, Inc.",Released
3,Jun 16 2023 1:27PM,264908,10,38552,"Digital Brands, LLC",Released
4,Jun 16 2023 1:27PM,264908,10,38553,"Digital Brands, LLC",Released
5,Jun 16 2023 1:26PM,264907,10,ALK39448,Alikai Health,Executing
6,Jun 16 2023 1:26PM,264907,10,ALK39471,Alikai Health,Released
7,Jun 16 2023 1:26PM,264907,10,ALK39483,Alikai Health,Executing
8,Jun 16 2023 1:26PM,264907,10,ALK39485,Alikai Health,Released
9,Jun 16 2023 1:26PM,264907,10,ALK39502,Alikai Health,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,264907,Released,9
13,264908,Released,2
14,264909,Released,1
15,264910,Released,1
16,264911,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264907,4.0,9.0
264908,NaN,2.0
264909,NaN,1.0
264910,NaN,1.0
264911,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264844,0.0,1.0
264845,0.0,1.0
264850,0.0,1.0
264866,1.0,0.0
264884,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264844,0,1
264845,0,1
264850,0,1
264866,1,0
264884,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264844,0,1
1,264845,0,1
2,264850,0,1
3,264866,1,0
4,264884,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264844,,1
1,264845,,1
2,264850,,1
3,264866,1,
4,264884,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2023 1:43PM,264911,16,Sartorius Bioprocess Solutions
1,Jun 16 2023 1:41PM,264910,10,"Nextsource Biotechnology, LLC"
2,Jun 16 2023 1:38PM,264909,10,"Methapharm, Inc."
3,Jun 16 2023 1:27PM,264908,10,"Digital Brands, LLC"
5,Jun 16 2023 1:26PM,264907,10,Alikai Health
18,Jun 16 2023 1:25PM,264906,19,"AdvaGen Pharma, Ltd"
20,Jun 16 2023 12:53PM,264903,10,"Senseonics, Incorporated"
22,Jun 16 2023 12:51PM,264902,20,"HVL, LLC dba Atrium Innovations"
23,Jun 16 2023 12:45PM,264901,18,Seqirus
24,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 16 2023 1:43PM,264911,16,Sartorius Bioprocess Solutions,,1
1,Jun 16 2023 1:41PM,264910,10,"Nextsource Biotechnology, LLC",,1
2,Jun 16 2023 1:38PM,264909,10,"Methapharm, Inc.",,1
3,Jun 16 2023 1:27PM,264908,10,"Digital Brands, LLC",,2
4,Jun 16 2023 1:26PM,264907,10,Alikai Health,4,9
5,Jun 16 2023 1:25PM,264906,19,"AdvaGen Pharma, Ltd",,2
6,Jun 16 2023 12:53PM,264903,10,"Senseonics, Incorporated",,2
7,Jun 16 2023 12:51PM,264902,20,"HVL, LLC dba Atrium Innovations",,1
8,Jun 16 2023 12:45PM,264901,18,Seqirus,1,
9,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd",34,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 1:43PM,264911,16,Sartorius Bioprocess Solutions,1,
1,Jun 16 2023 1:41PM,264910,10,"Nextsource Biotechnology, LLC",1,
2,Jun 16 2023 1:38PM,264909,10,"Methapharm, Inc.",1,
3,Jun 16 2023 1:27PM,264908,10,"Digital Brands, LLC",2,
4,Jun 16 2023 1:26PM,264907,10,Alikai Health,9,4
5,Jun 16 2023 1:25PM,264906,19,"AdvaGen Pharma, Ltd",2,
6,Jun 16 2023 12:53PM,264903,10,"Senseonics, Incorporated",2,
7,Jun 16 2023 12:51PM,264902,20,"HVL, LLC dba Atrium Innovations",1,
8,Jun 16 2023 12:45PM,264901,18,Seqirus,,1
9,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd",4,34


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 1:43PM,264911,16,Sartorius Bioprocess Solutions,1,
1,Jun 16 2023 1:41PM,264910,10,"Nextsource Biotechnology, LLC",1,
2,Jun 16 2023 1:38PM,264909,10,"Methapharm, Inc.",1,
3,Jun 16 2023 1:27PM,264908,10,"Digital Brands, LLC",2,
4,Jun 16 2023 1:26PM,264907,10,Alikai Health,9,4


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 1:43PM,264911,16,Sartorius Bioprocess Solutions,1.0,NaN
1,Jun 16 2023 1:41PM,264910,10,"Nextsource Biotechnology, LLC",1.0,NaN
2,Jun 16 2023 1:38PM,264909,10,"Methapharm, Inc.",1.0,NaN
3,Jun 16 2023 1:27PM,264908,10,"Digital Brands, LLC",2.0,NaN
4,Jun 16 2023 1:26PM,264907,10,Alikai Health,9.0,4.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 1:43PM,264911,16,Sartorius Bioprocess Solutions,1.0,0.0
1,Jun 16 2023 1:41PM,264910,10,"Nextsource Biotechnology, LLC",1.0,0.0
2,Jun 16 2023 1:38PM,264909,10,"Methapharm, Inc.",1.0,0.0
3,Jun 16 2023 1:27PM,264908,10,"Digital Brands, LLC",2.0,0.0
4,Jun 16 2023 1:26PM,264907,10,Alikai Health,9.0,4.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1854226,17.0,4.0
12,264866,0.0,1.0
16,264911,1.0,0.0
18,264901,0.0,1.0
19,794676,6.0,35.0
20,529752,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1854226,17.0,4.0
1,12,264866,0.0,1.0
2,16,264911,1.0,0.0
3,18,264901,0.0,1.0
4,19,794676,6.0,35.0
5,20,529752,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,17.0,4.0
1,12,0.0,1.0
2,16,1.0,0.0
3,18,0.0,1.0
4,19,6.0,35.0
5,20,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,12,Released,0.0
2,16,Released,1.0
3,18,Released,0.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Executing,4.0,1.0,0.0,1.0,35.0,0.0
Released,17.0,0.0,1.0,0.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Executing,4.0,1.0,0.0,1.0,35.0,0.0
1,Released,17.0,0.0,1.0,0.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Executing,4.0,1.0,0.0,1.0,35.0,0.0
1,Released,17.0,0.0,1.0,0.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()